In [ ]:
# import ipyleaflet as ipl 
# pridavali by sme postupne, ako by sa chyby objavovali
from folium import Map, TileLayer, LayerControl, PolyLine, FeatureGroup, Circle
from folium.plugins import MarkerCluster
import gpxpy

In [ ]:
# ukazat trasy/hradiste.gpx, chceme ho nacitat
def read_gpx(gfile, meters=0):
    gpx_file = open(gfile, 'r')
    gpx = gpxpy.parse(gpx_file)  # vrati GPX objekt, nizsie ukazat, co vsetko obsahuje
    gpx.simplify(meters)         # vidime efekt dole
    lat, lon = [], []
    # assume one track, one segment, data: lat, lon, elev
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                lat.append(point.latitude)
                lon.append(point.longitude)    
    return (lat, lon)

In [ ]:
lat, lon = read_gpx('trasy/hradiste.gpx')
len(lat)
slat, slon = read_gpx('trasy/hradiste.gpx', meters=2)
len(lat), len(slat)

In [ ]:
# aby trasa bola 'vycentrovana'
npts = len(slat)
center = (sum(slat) / npts, sum(slon) / npts)
# mapa
m = Map(location=center, tiles=None, # neskor pridame basemaps
        zoom_start=14, width=800, height=600, control_scale=True)
esri_sat = TileLayer(tiles='Esri.WorldImagery', name='Esri Sat')
mtb_map = TileLayer(tiles='MtbMap', name='Mtb Map CZ')
osm_map = TileLayer(tiles='OpenStreetMap.Mapnik', name='OpenStreet Map')


In [ ]:
# Trasy
loc = list(zip(lat, lon))
sloc = list(zip(slat, slon))
ori_track = PolyLine(locations=loc, weight=2, name='povodna', overlay=True)
sim_track = PolyLine(locations=sloc, weight=2, color='cyan', overlay=True)
# Body
points = MarkerCluster(name='GPS body', maxClusterRadius=40)
for location in sloc:
    Circle(location=location, color='red', radius=2, fill=True).add_to(points)

In [ ]:
esri_sat.add_to(m)
mtb_map.add_to(m)
osm_map.add_to(m)
trasa_pres = FeatureGroup(name='Trasa presne').add_to(m)
trasa_pribl = FeatureGroup(name='Trasa približne').add_to(m)
ori_track.add_to(trasa_pres)
sim_track.add_to(trasa_pribl)
points.add_to(m) 
m.add_child(LayerControl());

In [ ]:
m

__Našim cieľom je zobraziť len body, kde sme na trase fotili a po kliknutí na bod aj miniatúru fotky.__